In [1]:
def fc(x, scope, nh, *, init_scale=1.0, init_bias=0.0):
    with tf.variable_scope(scope):
        nin = x.get_shape()[1].value
        w = tf.get_variable("w", [nin, nh])
        b = tf.get_variable("b", [nh])
        return tf.nn.relu(tf.matmul(x, w)+b)


In [2]:
import tensorflow as tf;
import sklearn.datasets;
iris_ds = sklearn.datasets.load_iris(return_X_y=False)
x = tf.placeholder(tf.float32, shape=(10, 4))
y = tf.placeholder(tf.float32, shape=(10,3))

In [3]:
iris_ds = sklearn.datasets.load_iris(return_X_y=False)
#Printing the dataset
print (iris_ds)
print (iris_ds["data"].shape)
print (iris_ds["target"].shape)

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

In [4]:
from random import shuffle
import numpy as np;
iris_data = np.array(iris_ds["data"])
iris_label = np.array(iris_ds["target"])
length = len(iris_label)
iris_idx = list(range(length))
shuffle(iris_idx)
train_idx = iris_idx[:round(length*2/3)]
test_idx = iris_idx[round(length*2/3):]

train_data,train_label = iris_data[train_idx,:],iris_label[train_idx]
test_data,test_label = iris_data[test_idx,:],iris_label[test_idx]
print (len(train_data))
#Reshaping Dataset
train_data,train_label = train_data.reshape(10,-1,4),train_label.reshape(10,-1)
print (train_data.shape,train_label.shape)

100
(10, 10, 4) (10, 10)


In [5]:
def build_model(x):
    y1 = fc(x,nh=5,scope="fc1")
    y2 = fc(y1,nh=3,scope = "fc2")
    return tf.nn.softmax(y2)

y_hat = build_model(x)

squared_loss = tf.reduce_sum(tf.square(y-y_hat));
#For Calculating Accuracy
prediction = tf.round(y)
correct = tf.cast(tf.equal(prediction, y), dtype=tf.float32)
accuracy = tf.reduce_mean(correct)

In [6]:
#Optimizers
optimizer = tf.train.AdamOptimizer().minimize(squared_loss)

In [20]:


with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    #Training the model
    for epoch in range(10):
        for (dt,lbl) in zip(train_data,train_label):
            _opt,_acc,_loss= sess.run([optimizer,accuracy,squared_loss],feed_dict={x:dt,y:tf.keras.utils.to_categorical(lbl,3)})
            #print (_acc,_loss)

In [21]:
logs_path = 'C:\\Users\\jaley\\git\\sess2\\tmp_model.ckpt'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
